### 14장 모델 성능 향상시키기

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 와인데이터
df = pd.read_csv('./data/wine.csv', header=None)
X = df.iloc[:,0:12]
y = df.iloc[:,12]

# 학습셋, 테스트셋 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# 모델 구조 설정
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델이 저장되는 조건 설정
modelpath = "./data/model/all/{epoch:02d}-{val_accuracy:.4f}.keras"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 모델 실행
history = model.fit(X_train, y_train, epochs=2000, batch_size=500, validation_split=0.25, 
                    verbose=1, callbacks=[early_stopping_callback, checkpointer]) 

# 테스트 결과 출력
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

hist_df = pd.DataFrame(history.history)
hist_df

# val_loss, loss값들 저장
y_vloss = hist_df['val_loss']
y_loss = hist_df['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, "o", c='red', markersize=2, label='Testset_loss')
plt.plot(x_len, y_loss, "o", c='blue', markersize=2, label='Trainset_loss')

plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


### 15장 실제 데이터로 만들어 보는모델

1)주택가격 데이터

In [ ]:
df = pd.read_csv("./data/house_train.csv")
df

# 각 데이터가 어떤 유형으로 되어 있는지? - 정수,실수,오브젝트형 등등 다양함
df.dtypes

# 판다스의 get_dummies()를 이용해서 카테고리형 변수를 0, 1로 이루어진 변수로 change (원핫 인코딩)
df = pd.get_dummies(df)

# 결측치 대체
df = df.fillna(df.mean())

# 데이터 사이의 상관관계 저장
df_corr = df.corr()

# *집값과 관련이 큰 것부터 순서대로 저장
df_corr_sort = df_corr.sort_values('SalePrice', ascending=False)

# 집값을 제외한 나머지 열을 저장한다
cols_train = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
X_train_pre = df[cols_train]

# 집값을 저장
y = df['SalePrice'].values

# 학습:테스트=8:2
X_train, X_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

# 모델 구조 설정
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(1))
model.summary()

# 모델을 실행
model.compile(optimizer='adam', loss='mean_squared_error')

# 20번 이상 결과가 향상되지 않으면 자동으로 중단
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델의 이름 설정
modelpath = './data/model/Ch15-house.keras'

# 최적화된 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                               verbose=0, save_best_only=True)

# 실행 관련 설정을 하는 부분 - 전체의 20%를 검증셋으로 설정
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000,
                    batch_size=32, callbacks=[early_stopping_callback, checkpointer])


In [ ]:
# 주택 가격 예측 모델
## 학습 결과를 시각화하기 위해 예측 값과 실제 값, 실행 번호가 들어갈 빈 리스트를 만들고,
## 25개의 샘플로부터 얻은 결과를 채워 넣는다
real_prices = []
pred_prices = [] # 예상가
X_num = []

n_iter = 0
Y_prediction = model.predict(X_test).flatten()

for i in range(25):
    real = y_test[i]
    prediction = Y_prediction[i]
    print('실제가격: {:.2f}, 예상가격: {:.2f}'.format(real, prediction))
    real_prices.append(real)

    pred_prices.append(prediction)
    n_iter += 1
    X_num.append(n_iter)

# 이제 위 결과를 그래프를 통해 비교해보자
plt.plot(X_num, pred_prices, label='predicted price')
plt.plot(X_num, real_prices, label='real price')
plt.legend()
plt.show()

2)추가데이터 - car

In [ ]:
train_df = pd.read_excel('https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true', sheet_name='train')
test_df = pd.read_excel('https://github.com/cranberryai/todak_todak_python/blob/master/machine_learning/regression/carprice_E1SUl6b.xlsx?raw=true', sheet_name='test')

### 16장 이미지 인식의 꽃, 컨볼루션신경망(CNN)

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sys

# MNIST 데이터셋을 불러와 학습&테스트셋으로 저장
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 학습셋과 테스트셋이 각각 몇 개의 이미지로 되어 있는지 화깅ㄴ
print('학습셋 이미지 수: %d개' % (X_train.shape[0]))
print('테스트 이미지 수: %d개' % (X_test.shape[0]))

# 첫 번째 이미지를 확인
plt.imshow(X_train[0], cmap='Greys')
plt.show()

# 이미지가 인식되는 원리
for x in X_train[0]:
    for i in x:
        sys.stdout.write('%-3s' % i)
    sys.stdout.write('\n')

# 차원 변환 과정을 연습해보자
X_train = X_train.reshape(X_train.shape[0], 784)
X_train = X_train.astype('float32')
X_train = X_train/255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

# 클래스값을 확인
print('class: %d' % (y_train[0]))

# 바이너리화 과정을 연습해보자
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(y_train[0])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# 딥러닝 모델 기본 프레임
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax')) # 이건 10개 output - 0~9까지 [0,0,0,0,0,1,0,0,0,0]
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # 최적화함수는 adam

# 모델 최적화를 위한 설정 구간
modelpath = './data/MNIST_MLP.keras'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델 실행
history = model.fit(X_train, y_train, validation_split=0.25, epochs=30, batch_size=200,
                    verbose=0, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력
print('\n Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

# 검증셋과 학습셋의 오차를 저장 - 그래프로 보여줄거임
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 실행 결과를 그래프로 표현해 보자. 이번에는 학습셋의 정확도 대신 학습셋의 오차를 그래프로 표현
# 학습셋의 오차는 1에서 학습셋의 정확도를 뺀 값
# 좀 더 세밀한 변화를 볼 수 있게 학습셋의 오차와 테스트셋의 오차를 그래프 하나로 나타내겠음